In [2]:
import os
import traceback
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import sqlalchemy
import matplotlib.pyplot as plt
import numpy as np

user='hifld_summer'
pw='hifld_summer'
name='hifld_summer'
host='moria'
port=2023
engine = sqlalchemy.create_engine(
    f'postgresql://{user}:{pw}@{host}:{port}/{name}'
    )

In [13]:
sql="""
select geom, build_id, prim_occ from structures.alabama limit 500000
"""
try:
    gdf = gpd.GeoDataFrame.from_postgis(sql, con=engine,geom_col='geom') # epsg 4346

    gdf = gdf.to_crs("ESRI:102003") # USA_Contiguous_Albers_Equal_Area_Conic Projection

    gdf["length"] = gdf.bounds["maxy"] - gdf.bounds["miny"]
    gdf["width"] = gdf.bounds["maxx"] - gdf.bounds["minx"]

    gdf["long_side"] = gdf[["width", "length"]].max(axis=1)
    gdf["short_side"] = gdf[["width", "length"]].min(axis=1)

    gdf["ls_ratio"] = gdf["long_side"] / gdf["short_side"]
    gdf["centroid"] = gdf.geometry.centroid
    buffer_distance = 250
    gdf["buffer_geom"] = gdf.centroid.buffer(buffer_distance)
    buffer_distance = str(buffer_distance)

    buffers_gdf = gdf.copy()
    centroids_gdf = gdf.copy()
    
    buffers_gdf = buffers_gdf.set_geometry('buffer_geom')
    centroids_gdf = gdf.set_geometry('centroid')

    buffers_gdf.rename(columns = {'build_id':'build_id_right'}, inplace = True)
    centroids_gdf.rename(columns = {'build_id':'build_id_left'}, inplace = True)

    
    buffers_gdf = buffers_gdf.drop(columns=['centroid', 'geom', 'ls_ratio'])
    centroids_gdf = centroids_gdf.drop(columns=['buffer_geom', 'geom'])

    def mean_ls_ratio_of_geoms_in_buffers(points, polygons, polygon_id, new_column="mean_area"):
      
        points_in_polygon = (polygons.sjoin(points, how="inner").groupby(polygon_id)["ls_ratio"].mean().rename(new_column))
        polygons = (polygons.set_index(polygon_id).join(points_in_polygon, how="left").fillna({new_column: 0}))
    
        return polygons

    neighbor_gdf = mean_ls_ratio_of_geoms_in_buffers(centroids_gdf, buffers_gdf, "build_id_right", new_column="mean_ls_ratio_of_neighbors")

except Exception:
    print(traceback.print_exc())

count    5000.000000
mean        1.422010
std         0.211834
min         1.052553
25%         1.279865
50%         1.375257
75%         1.505387
max         3.070259
Name: mean_ls_ratio_of_neighbors, dtype: float64
